In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

C:\Users\Iker\AppData\Local\Temp\ipykernel_29508\1756040219.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
total_data = pd.read_csv("TotalDataset_Engineered.csv") 

In [3]:
total_data.tail(5)

,Unnamed: 0,Date first seen,Duration,Proto,Src IP Addr,Src Pt,Dst IP Addr,Dst Pt,Packets,Bytes,...,S,F,class,attackType,year,month,days,hours,minutes,seconds
39643004,39643004,2019-03-06 21:18:57,0.0,UDP,216.239.36.10,53,158.129.192.2,48598.0,2,290,...,0,0,normal,none,2019,3,6,21,18,57.0
39643005,39643005,2019-03-06 21:18:57,0.0,UDP,194.0.4.10,53,192.71.30.8,34617.0,2,268,...,0,0,normal,none,2019,3,6,21,18,57.0
39643006,39643006,2019-03-06 21:18:57,0.0,UDP,83.171.8.8,53,31.13.112.156,19981.0,2,136,...,0,0,normal,none,2019,3,6,21,18,57.0
39643007,39643007,2019-03-06 21:18:57,0.0,UDP,192.71.30.8,34617,194.0.4.10,53.0,2,138,...,0,0,normal,none,2019,3,6,21,18,57.0
39643008,39643008,2019-03-06 21:18:57,0.0,UDP,2620:0:cc9::70,27547,2001:678:6::1,53.0,2,174,...,0,0,normal,none,2019,3,6,21,18,57.0


Change variables types.

In [4]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39643009 entries, 0 to 39643008
Data columns (total 24 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Unnamed: 0       int64  
 1   Date first seen  object 
 2   Duration         float64
 3   Proto            object 
 4   Src IP Addr      object 
 5   Src Pt           int64  
 6   Dst IP Addr      object 
 7   Dst Pt           float64
 8   Packets          int64  
 9   Bytes            int64  
 10  U                int64  
 11  A                int64  
 12  P                int64  
 13  R                int64  
 14  S                int64  
 15  F                int64  
 16  class            object 
 17  attackType       object 
 18  year             int64  
 19  month            int64  
 20  days             int64  
 21  hours            int64  
 22  minutes          int64  
 23  seconds          float64
dtypes: float64(3), int64(15), object(6)
memory usage: 7.1+ GB


In [5]:
total_data['year'] = total_data['year'].astype('int64')
total_data['month'] = total_data['month'].astype('int64')
total_data['days'] = total_data['days'].astype('int64')
total_data['hours'] = total_data['hours'].astype('int64')
total_data['minutes'] = total_data['minutes'].astype('int64')
total_data['seconds'] = total_data['seconds'].astype('int64')
total_data['Dst Pt'] = total_data['Dst Pt'].astype('int64')
total_data['Proto'] = total_data['Proto'].astype('category')

In [6]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39643009 entries, 0 to 39643008
Data columns (total 24 columns):
 #   Column           Dtype   
---  ------           -----   
 0   Unnamed: 0       int64   
 1   Date first seen  object  
 2   Duration         float64 
 3   Proto            category
 4   Src IP Addr      object  
 5   Src Pt           int64   
 6   Dst IP Addr      object  
 7   Dst Pt           int64   
 8   Packets          int64   
 9   Bytes            int64   
 10  U                int64   
 11  A                int64   
 12  P                int64   
 13  R                int64   
 14  S                int64   
 15  F                int64   
 16  class            object  
 17  attackType       object  
 18  year             int64   
 19  month            int64   
 20  days             int64   
 21  hours            int64   
 22  minutes          int64   
 23  seconds          int64   
dtypes: category(1), float64(1), int64(17), object(5)
memory usage: 6.8+ GB


Categorizing the Proto as Mark did:

In [64]:
proto_enc_df = pd.get_dummies(total_data['Proto'], dtype='int64')

total_data = .join(proto_enc_df)

In [65]:
total_data['class'] = total_data['class'].map(
    {"normal":0,"victim":1})

In [66]:
total_data['class'].unique()

array([1, 0], dtype=int64)

Standarization is not needed for Random Forest application.

In [67]:
features = list(total_data.columns)
features.remove("Unnamed: 0")
features.remove("Date first seen")
features.remove('Src IP Addr')
features.remove('Dst IP Addr')
features.remove('Proto')
features.remove("class")
features.remove("attackType")


print(features)


['Duration', 'Src Pt', 'Dst Pt', 'Packets', 'Bytes', 'U', 'A', 'P', 'R', 'S', 'F', 'year', 'month', 'days', 'hours', 'minutes', 'seconds', 'AH', 'ESP', 'GRE', 'ICMP', 'ICMP6', 'IGMP', 'IPIP', 'IPv6', 'OSPF', 'PIM', 'TCP', 'UDP', 'VRRP']


Let's create the target:

In [106]:
total_data['class'].value_counts()
total_data['Anomaly'] = total_data['class']

KeyError: "['class'] not found in axis"

In [115]:
total_data = total_data.drop('class',axis = 1)

Using the same splitting class as MARK:

In [87]:
class Split():
    """
    Mark's Split class
    """
    
    def __init__(self, num = 5):  # num refers to the number of datasets you wanna split the original total dataset into
        self.total_data = total_data
        self._0 = []
        self._1 = []
        self.cv = num

    def generate(self):
        self._split()

        res = []
        
        for i in tqdm(range(self.cv)):
            if i != self.cv-1:
                tmp = self._0[i*(len(self._0)//self.cv):(i+1)*(len(self._0)//self.cv)] + self._1[i*(len(self._1)//self.cv):(i+1)*(len(self._1)//self.cv)]
            elif i == self.cv-1:
                tmp = self._0[i*(len(self._0)//self.cv):-1] + self._1[i*(len(self._1)//self.cv):-1]
            res.append(tmp)

        return res # Returns arrays of indices
        
    def _split(self):
        length = len(self.total_data)
        print("Splitting...")
        for i in tqdm(range(length)):
            ttype = self.total_data.iloc[i].Anomaly
            if ttype == 0:
                    self._0.append(i)
            else:
                    self._1.append(i)

        print("Splitting done!")

In [88]:
split = Split(num = n_subset) # num refers to the number of datasets you wanna split the original total dataset into
split_data = split.generate()

Splitting...


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1718.79it/s]


Splitting done!


100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 3166.95it/s]


In [204]:
train_validation_data = split_data[0] + split_data[1] + split_data[2] + split_data[3]
test_validation_data = split_data[-1]

In [187]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

import time

# Define hyperparameters list for RandomForest
hyperparameters = {
    'max_depth': [None, 10, 20],
    'n_estimators': [50, 100, 200]
}

# Generate all combinations of hyperparameters
param_combinations = ParameterGrid(hyperparameters)

# Training various different models for reliability
models = [RandomForestClassifier(**params, random_state = 0) for params in param_combinations]

# Batch Training our model
testing_subset_number = n_subset-1
total_subset_arr = [i for i in range(n_subset)]
total_subset_arr.remove(testing_subset_number) #removing last subset

acc = np.zeros((len(models),n_subset))

for subset in total_subset_arr:  # Testing on the last subset
    print("Subset {}".format(subset + 1))
    print("----------------------")

    # Obtaining our i-th subset of the total data
    X = total_data.iloc[split_data[subset]][features]
    y = total_data.iloc[split_data[subset]]['Anomaly']

    # Applying StratifiedKFold to the specific subset
    skf = StratifiedKFold(n_splits=len(models))
    skf.get_n_splits(X, y)

    for i, (train_index, test_index) in enumerate(skf.split(X, y)):
        start_time = time.time()
        print(f"\nSubset - {subset + 1} | Fold - {i + 1}")
        print("----------------------")

        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Paste models here
        model = models[i]

        # Train
        if subset == 0:
            print("Training...")
            print("Using model with parameters :"+str(param_combinations[i]))
            print("----------------------")
            model.fit(X_train, y_train)
        else:
            print("Training using saved model  :"+str(param_combinations[i]))
            print("----------------------")  # Batch Training
            model.fit(X_train, y_train)

        print("Training done!")
        print("----------------------")

        models.append(model)

        # Predict
        pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, pred)
        #acc.append(accuracy)

        # Printing
        #print("Subset - {} | Fold - {} Accuracy: {:.2f}%".format(subset + 1, i + 1, accuracy * 100))
        print("Time taken: {:.0f}s".format(time.time() - start_time))
        print("----------------------")

    print("------+++++++--------")
    print("------+++++++--------")
    print("                                                 Average accuracy: {:.2f}%".format(np.mean(acc) * 100))
    print("------+++++++--------")
    print("------+++++++--------")

Subset 1
----------------------
200

Subset - 1 | Fold - 1
----------------------
Training...
Using model with parameters :{'n_estimators': 50, 'max_depth': None}
----------------------
Training done!
----------------------
Time taken: 0s
----------------------

Subset - 1 | Fold - 2
----------------------
Training...
Using model with parameters :{'n_estimators': 100, 'max_depth': None}
----------------------
Training done!
----------------------
Time taken: 0s
----------------------

Subset - 1 | Fold - 3
----------------------
Training...
Using model with parameters :{'n_estimators': 200, 'max_depth': None}
----------------------
Training done!
----------------------
Time taken: 1s
----------------------

Subset - 1 | Fold - 4
----------------------
Training...
Using model with parameters :{'n_estimators': 50, 'max_depth': 10}
----------------------
Training done!
----------------------
Time taken: 0s
----------------------

Subset - 1 | Fold - 5
----------------------
Training...
Us

IndexError: ParameterGrid index out of range

In [220]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'max_features':[1,3,5,7],
              'min_samples_leaf':[2,3],
              'min_samples_split':[2,3]
               }

testing_subset_number = n_subset-1
total_subset_arr = [i for i in range(n_subset)]
total_subset_arr.remove(testing_subset_number) #removing last subset

list_rf = []

for subset in total_subset_arr:  # Testing on the last subset
    print("Subset {}".format(subset + 1))
    print("----------------------")

    # Obtaining our i-th subset of the total data
    X = total_data.iloc[split_data[subset]][features]
    y = total_data.iloc[split_data[subset]]['Anomaly']

   

    rf = RandomForestClassifier()
    # Random search of parameters, using 3 fold cross validation, 
    # search across 100 different combinations, and use all available cores
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
    # Fit the random search model
    rf_random.fit(X, y)

    list_rf.append(rf_random)

    print('Best hyperparameters are: '+str(rf_random.best_params_))
    print('Best score is: '+str(rf_random.best_score_))


    

Subset 1
----------------------
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best hyperparameters are: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 7, 'max_depth': 10}
Best score is: 0.8800000000000001
Subset 2
----------------------
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best hyperparameters are: {'n_estimators': 1200, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_features': 1, 'max_depth': 40}
Best score is: 0.9400000000000001
Subset 3
----------------------
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best hyperparameters are: {'n_estimators': 600, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 1, 'max_depth': 40}
Best score is: 0.9
Subset 4
----------------------
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best hyperparameters are: {'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_features': 7, 'max_depth': 60}
Best sc

In [ ]:
Xpp= X_train
ypp= y_train


rfpp = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_randompp = RandomizedSearchCV(estimator = rfpp, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_randompp.fit(Xpp, ypp)

In [227]:
print('Best hyperparameters are: '+str(rf_randompp.best_params_))
print('Best score is: '+str(rf_randompp.best_score_))

Best hyperparameters are: {'n_estimators': 2000, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_features': 5, 'max_depth': 10}
Best score is: 0.9087500000000001


In [228]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    accuracy = accuracy_score(test_labels, predictions)
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.5f}%.'.format(accuracy))
    
    return accuracy

X_train = total_data.iloc[train_validation_data][features]
y_train = total_data.iloc[train_validation_data]['Anomaly']
X_test = total_data.iloc[test_validation_data][features]
y_test = total_data.iloc[test_validation_data]['Anomaly']

base_model = RandomForestClassifier(random_state = 0)
base_model.fit(X_train, y_train)
print("BASE MODEL")
base_accuracy = evaluate(base_model, X_test, y_test)
print('_______________________')
print('Base Accuracy = {:0.2f}%.'.format(base_accuracy))

model1 = RandomForestClassifier(**list_rf[0].best_params_, random_state = 0)
print(list_rf[0].best_params_)
model1.fit(X_train, y_train)
model1_accuracy = evaluate(model1,X_test,y_test)

model2 = RandomForestClassifier(**list_rf[1].best_params_, random_state = 0)
print(list_rf[1].best_params_)
model2.fit(X_train, y_train)
model2_accuracy = evaluate(model2,X_test,y_test)

model3 = RandomForestClassifier(**list_rf[2].best_params_, random_state = 0)
print(list_rf[2].best_params_)
model3.fit(X_train, y_train)
model3_accuracy = evaluate(model3,X_test,y_test)

model4 = RandomForestClassifier(**list_rf[3].best_params_, random_state = 0)
print(list_rf[3].best_params_)
model4.fit(X_train, y_train)
model4_accuracy = evaluate(model4,X_test,y_test)

model5 = RandomForestClassifier(**rf_randompp.best_params_, random_state = 0)
print(rf_randompp.best_params_)
model5.fit(X_train, y_train)
model5_accuracy = evaluate(model4,X_test,y_test)

BASE MODEL
Model Performance
Average Error: 0.0960 degrees.
Accuracy = 0.90404%.
_______________________
Base Accuracy = 0.90%.
{'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 7, 'max_depth': 10}
Model Performance
Average Error: 0.0909 degrees.
Accuracy = 0.90909%.
{'n_estimators': 1200, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_features': 1, 'max_depth': 40}
Model Performance
Average Error: 0.0960 degrees.
Accuracy = 0.90404%.
{'n_estimators': 600, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 1, 'max_depth': 40}
Model Performance
Average Error: 0.0960 degrees.
Accuracy = 0.90404%.
{'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_features': 7, 'max_depth': 60}
Model Performance
Average Error: 0.0909 degrees.
Accuracy = 0.90909%.
{'n_estimators': 2000, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_features': 5, 'max_depth': 10}
Model Performance
Average Error: 0.0909 degrees.
Accuracy = 0.90